In [3]:
!pip install anndata


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip


In [4]:
import pandas as pd

bulk_feature_counts = pd.read_csv("/Users/kpax/Documents/aep/study/MSC/lab/PPMI_Project_133_RNASeq/ppmi_counts_meta_dataset.csv", index_col=0)

In [5]:
patient_status = pd.read_csv("/Users/kpax/Documents/aep/study/MSC/lab/PPMI_Project_133_RNASeq/Participant_Status_19Mar2025.csv")

In [6]:
genetic_mapping = {
    'ENRLPINK1': 'PINK1',
    'ENRLPRKN': 'PRKN',
    'ENRLSRDC': 'SRDC',
    'ENRLHPSM': 'HPSM',
    'ENRLRBD': 'RBD',
    'ENRLLRRK2': 'LRRK2',
    'ENRLSNCA': 'SNCA',
    'ENRLGBA': 'GBA'
}

def get_genetic_group(row):
    for col, group in genetic_mapping.items():
        if row[col] == 1:
            return group
    return 'None'

genetic_groups = patient_status[['PATNO'] + list(genetic_mapping.keys())].copy()
genetic_groups['Genetic_Group'] = genetic_groups.apply(get_genetic_group, axis=1)

bulk_feature_counts = bulk_feature_counts.merge(
    genetic_groups[['PATNO', 'Genetic_Group']],
    left_on='Patient',
    right_on='PATNO',
    how='left'
).set_index(bulk_feature_counts.index)

bulk_feature_counts['Genetic_Group'] = bulk_feature_counts['Genetic_Group'].fillna('Unknown')

In [7]:
def map_age_group(age):
    if 30 <= age <= 50:
        return "30-50"
    elif 50 < age <= 70:
        return "50-70"
    elif 70 < age <= 80:
        return "70-80"
    elif age > 80:
        return ">80"
    return "Unknown"

patient_status['Age_Group'] = patient_status['ENROLL_AGE'].apply(map_age_group)

age_group_mapping = patient_status[['PATNO', 'Age_Group']]

bulk_feature_counts = bulk_feature_counts.merge(
    age_group_mapping,
    left_on='Patient',
    right_on='PATNO',
    how='left'
).set_index(bulk_feature_counts.index)

In [8]:
import re
counts_raw = bulk_feature_counts.iloc[:,bulk_feature_counts.columns.str.startswith("ENSG")]
counts_raw_t = counts_raw.T
trunc_eid = [re.sub(r"\.\d+$", "", eid) for eid in counts_raw_t.index.values.tolist()]

In [11]:
!pip install mygene


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip


In [12]:
import mygene

mg = mygene.MyGeneInfo()
mappings = mg.querymany(
    trunc_eid,
    scopes="ensembl.gene",
    fields="symbol",
    species="human"
)
import pandas as pd
df = pd.DataFrame(mappings)
df = df[['query', 'symbol']].rename(columns={'query': 'Ensembl_ID', 'symbol': 'Gene_Symbol'})
df['Gene_Symbol'] = df['Gene_Symbol'].fillna(df['Ensembl_ID'])
dup_counts = df['Ensembl_ID'].value_counts()
print(f"Total duplicates: {len(dup_counts[dup_counts > 1])}")
print(dup_counts[dup_counts > 1])

df_deduped = df.drop_duplicates(subset=['Ensembl_ID'], keep='first')
counts_raw_export = counts_raw_t.reset_index(drop=False)
counts_raw_export['index'] = trunc_eid
counts_raw_export = counts_raw_export.rename(columns={'index':'Ensembl_ID'})
counts_raw_export = (
    counts_raw_export.merge(df_deduped, on='Ensembl_ID', how='left')
    .set_index('Gene_Symbol')
    .drop(columns=['Ensembl_ID'])
)


Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed
31 input query terms found dup hits:	[('ENSG00000228044', 2), ('ENSG00000226506', 2), ('ENSG00000261600', 2), ('ENSG00000234162', 2), ('E
1428 input query terms found no hit:	['ENSG00000238009', 'ENSG00000230699', 'ENSG00000241180', 'ENSG00000236948', 'ENSG00000277726', 'ENS


Total duplicates: 31
Ensembl_ID
ENSG00000275757    10
ENSG00000273730    10
ENSG00000275405     4
ENSG00000276241     4
ENSG00000277927     3
ENSG00000268674     3
ENSG00000278294     3
ENSG00000285761     3
ENSG00000278932     3
ENSG00000262352     3
ENSG00000268595     2
ENSG00000275496     2
ENSG00000280018     2
ENSG00000259182     2
ENSG00000226519     2
ENSG00000257545     2
ENSG00000228566     2
ENSG00000284116     2
ENSG00000234352     2
ENSG00000188660     2
ENSG00000233656     2
ENSG00000249738     2
ENSG00000215156     2
ENSG00000250406     2
ENSG00000243620     2
ENSG00000227110     2
ENSG00000282304     2
ENSG00000234162     2
ENSG00000261600     2
ENSG00000226506     2
ENSG00000228044     2
Name: count, dtype: int64


In [13]:
print(counts_raw_export.shape)
print(bulk_feature_counts.shape)
print(counts_raw_export.head())

(58780, 4756)
(4756, 58789)
             3174.V08.0000373111.5104.SL.0001  \
Gene_Symbol                                     
DDX11L1                                    15   
WASH7P                                    232   
MIR6859-1                                  15   
MIR1302-2HG                                 0   
MIR1302-2                                   0   

             3201.BL.PP0016.0526.5104.SL.0002  \
Gene_Symbol                                     
DDX11L1                                     2   
WASH7P                                    199   
MIR6859-1                                  10   
MIR1302-2HG                                 0   
MIR1302-2                                   0   

             3057.V06.0003125874.5104.SL.0003  \
Gene_Symbol                                     
DDX11L1                                     0   
WASH7P                                    148   
MIR6859-1                                   2   
MIR1302-2HG                            

In [14]:
counts_raw_export.index.name = None
counts_raw_export.head()

,3174.V08.0000373111.5104.SL.0001,3201.BL.PP0016.0526.5104.SL.0002,3057.V06.0003125874.5104.SL.0003,4126.V04.0003179818.5104.SL.0004,4117.BL.0003164601.5104.SL.0005,3057.V04.0003119973.5104.SL.0006,3471.BL.PP0015.6713.5104.SL.0007,3174.V04.0003162443.5104.SL.0008,3661.V04.0003169393.5104.SL.0009,3551.V06.0003158673.5104.SL.0010,...,41375.V02.0003314306.5104.SL.4885,41292.V04.0003314317.5104.SL.4886,41356.V02.0003314328.5104.SL.4887,41296.V02.0003314372.5104.SL.4888,41294.V04.0003314383.5104.SL.4889,41314.V02.0003314545.5104.SL.4890,42308.V02.0003376702.5104.SL.4891,42306.V02.0003376763.5104.SL.4892,1009.POOL.0003203630.5104.SL.4893,1010.POOL.0003202476.5104.SL.4894
DDX11L1,15,2,0,6,0,17,0,8,0,1,...,1,4,7,3,8,10,20,6,4,6
WASH7P,232,199,148,216,95,112,97,199,232,209,...,59,160,386,114,111,215,217,142,147,182
MIR6859-1,15,10,2,16,3,13,8,0,7,5,...,9,32,39,6,24,14,54,48,21,16
MIR1302-2HG,0,0,1,0,0,3,0,1,0,0,...,0,0,0,0,0,0,1,1,0,0
MIR1302-2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
ensembl_symbol_mapping = pd.DataFrame({
    "gene_symbol": counts_raw_export.index,
    "ensembl_id": counts_raw_t.index,
    "trunc_eid": trunc_eid
}).set_index('ensembl_id')

In [16]:
import anndata as ad
import numpy as np

ppmi_adata = ad.AnnData(bulk_feature_counts.loc[:, bulk_feature_counts.columns.str.startswith("ENSG")])
ppmi_adata.obs['Sample'] = bulk_feature_counts['Sample'].values
ppmi_adata.obs['Diagnosis'] = bulk_feature_counts['Diagnosis'].values
ppmi_adata.obs['Visit'] = bulk_feature_counts['Visit'].values
ppmi_adata.obs['Gender'] = bulk_feature_counts['Gender'].values
ppmi_adata.obs['Patient'] = bulk_feature_counts['Patient'].values
ppmi_adata.obs['Genetic_Group'] = bulk_feature_counts['Genetic_Group'].values
ppmi_adata.obs['Age_Group'] = bulk_feature_counts['Age_Group'].values
ppmi_adata.layers['counts_log2'] = np.log2(ppmi_adata.X + 1)
ppmi_adata.layers['gene_symbols'] = counts_raw_export.T
ppmi_adata.varm['symbol_ensembl_mapping'] = ensembl_symbol_mapping

In [1]:
!pip install graphviz


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip


In [20]:
from graphviz import Digraph

def plot_anndata_structure(adata, filename="anndata_structure", format="png"):
    dot = Digraph()
    dot.node("AnnData", "AnnData")

    # Add slots (obs, var, obsm, etc.)
    for slot in ["obs", "var", "obsm", "varm", "layers", "uns"]:
        dot.node(slot, slot)
        dot.edge("AnnData", slot)

        # Add keys inside each slot
        content = getattr(adata, slot)
        if isinstance(content, dict) or hasattr(content, 'keys'):
            for key in content.keys():
                dot.node(f"{slot}_{key}", key)
                dot.edge(slot, f"{slot}_{key}")

    # Render and save
    dot.render(filename=filename, format=format, cleanup=True)
    print(f"Saved to {filename}.{format}")

# Usage
plot_anndata_structure(ppmi_adata, filename="anndata_structure", format="png")

Saved to anndata_structure.png


In [41]:

ppmi_adata.write_h5ad("/Users/kpax/Documents/aep/study/MSC/lab/PPMI_Project_133_RNASeq/ppmi_adata.h5ad")